## Решение задачи линейного программирования симплекс-методом

In [1]:
import copy

class Simplex_table(object):
    
    def __init__(self, A):
        
        self.A = copy.deepcopy(A)
        self.index_column = []
        self.index_line = ['S_i0']
        self.A_tmp = copy.deepcopy(A)
        self.length = len(self.A[0])
        self.height = len(self.A)
        self.service_column = 0
        self.service_line = 0
        self.minimal_ratio = max(A[0])**2
        self.tmp = 0
        self.free_sm_zero = False           #True if there is a negative elem in independent column
        self.base_sol = False               #True if there is a possible base solution
        self.being_serv_col = False         #True if there is a service column
        self.being_serv_line = False        #True if there is a service line
        self.no_sol = False                 #True if there is no solution at all
        self.pos_in_func = False            #True if there is positive coef in the function's line
        self.optimal_sol = False            #True if the solution is optimal
        
    def Jordan_eliminations(self, serv_line, serv_col):
        
        self.A_tmp = copy.deepcopy(self.A)
        for line in range(self.height):
            for col in range(self.length):
                
                if (col == serv_col) and (line != serv_line):
                    self.A_tmp[line][col] = - (self.A[line][col] / self.A[serv_line][serv_col])
                    
                elif (col != serv_col) and (line == serv_line):
                    self.A_tmp[line][col] = self.A[line][col] / self.A[serv_line][serv_col]
                
                elif (col != serv_col) and (line != serv_line):
                    self.A_tmp[line][col] = self.A[line][col] - \
                    (self.A[line][serv_col]*self.A[serv_line][col]/self.A[serv_line][serv_col])
                    
        self.A_tmp[serv_line][serv_col] = 1/self.A[serv_line][serv_col]
        self.A = copy.deepcopy(self.A_tmp)
        tmp = self.index_line[self.service_column]
        self.index_line[self.service_column] = self.index_column[self.service_line]
        self.index_column[self.service_line] = tmp
        del tmp
            
        
    def find_base_solution(self):
                
        print("----Поиск базисного решения----")
        
        self.base_sol = False
        
        while self.base_sol == False:
            
            self.minimal_ratio = max(self.A[0])**2
            
            self.free_sm_zero = False
            self.being_serv_col = False
            self.being_serv_line = False
            self.no_sol = False
            self.view()
            
            for it1 in range(len(self.A) - 1):
                
                if self.A[it1][0] < 0:
                    
                    self.free_sm_zero = True
                    for it2 in range(1,self.length):
                        
                        if self.A[it1][it2] < 0:
                            self.being_serv_col = True
                            self.service_column = it2
                            break;
                            
            if self.free_sm_zero and (self.being_serv_col == False):
                self.no_sol = True
                
            if self.free_sm_zero == False:
                self.base_sol = True
                print('---Базисное решение:---')
                self.view()
                break;
            
            for it1 in range(self.height - 1):
                
                self.tmp = self.A[it1][0] / self.A[it1][self.service_column]
                if ( self.tmp < self.minimal_ratio) and (self.tmp > 0):
                    self.being_serv_line = True
                    self.minimal_ratio = self.tmp
                    self.service_line = it1
                    
            if self.free_sm_zero and (self.being_serv_line == False):
                self.no_sol = True
                
            if self.no_sol:
                return 'Нет решений.'
            print('Целевой столбец:', self.index_line[self.service_column])
            print('Целевая строка:', self.index_column[self.service_line])
            self.Jordan_eliminations(self.service_line,self.service_column)
    
    def find_optimal_solution(self):
        
        
        for it1 in range(1, self.length):
            
            if self.A[self.height - 1][it1] > 0:
                self.pos_in_func = True
                
        print("----Поиск оптимального решения----")
                
        while self.pos_in_func == True:
            
            self.minimal_ratio = max(self.A[0])**2
            
            self.view()
            
            for it1 in range(1, self.length):
            
                if self.A[self.length - 1][it1] > 0:
                    self.service_column = it1
                    break
            
           
            for it1 in range(self.height - 1):
                if self.A[it1][self.service_column] != 0:
                    
                    self.tmp = self.A[it1][0] / self.A[it1][self.service_column]
                    if ( self.tmp < self.minimal_ratio) and (self.tmp > 0):
                        self.being_serv_line = True
                        self.minimal_ratio = self.tmp
                        self.service_line = it1
                        
            
            print('Целевой столбец:', self.index_line[self.service_column])
            print('Целевая строка:', self.index_column[self.service_line])
        
            self.Jordan_eliminations(self.service_line,self.service_column)
            
            self.pos_in_func = False
            
            
            for it1 in range(1, self.length):
            
                if self.A[self.height - 1][it1] > 0:
                    self.pos_in_func = True
        
    def set_indexes(self,C,L):
        for i in C:
            self.index_column.append('x'+str(i))
        self.index_column.append('F')
        for i in L:
            self.index_line.append('x' + str(i))
                
    def view(self):
        
        print('*'*45)
        
        print('    ',end=' ')                       # Отступ в зависимости от len столбца
        l=5
        for it1 in self.index_line:
            print(it1, ' '*(7-len(it1)), end = ' ')
            l+=8
        i=0
        print()
        print('-'*l)
        print()
        for it1 in self.A:
            print(self.index_column[i],' '*(3-len(self.index_column[i])), end=' ')
            i+=1
            for it2 in range(len(it1)):
                
                print(' ',it1[it2],' '*(5-len(str(it1[it2]))), end=' ')
            print()
            print('-'*l)
            print()
        print('*'*45)
        del l

In [2]:
A = [[5,1,0,1],[1,3,1,-1],[2.5,-0.5,1,-0.5],[25,4,-5,5]] # неравенства-ограничения и целевая функция
Col = [2,4,6] # разметка таблицы
Lin = [1,3,5]

### Теоретические сведения

В предыдущем параграфе для решения ОЗЛП был применен графический метод. В общем случае ОЗЛП решается с помощью симплекс-метода, к изложению которого мы сейчас и переходим.
    
&#8195;&#8195; Рассмотрим ОЗЛП с системой ограничений в следующей форме:
   
   \begin{equation*}
       \begin{cases}
           {a_{11}x_1+a_{12}x_2+...+a_{1n}x_n}\leq b_1,\\
           {a_{21}x_1+a_{22}x_2+...+a_{2n}x_n}\leq b_2,\\
           ...............................\\
           {a_{m1}x_1+a_{m2}x_2+...+a_{mn}x_n}\leq b_m,
       \end{cases}
   \end{equation*}
   
&#8195;&#8195; Если ввести в систему ограничений дополнительные переменные:
   
&#8195;&#8195; $x_{n+1}$, $x_{n+2}$, ..., $x_{n+m}$
    
&#8195;&#8195; по формулам:
    
   \begin{equation*}
       \begin{cases}
           x_{n+1}=b_1-{a_{11}x_1+a_{12}x_2+...+a_{1n}x_n}\\
           x_{n+2}=b_2-{a_{21}x_1+a_{22}x_2+...+a_{2n}x_n}\\
            ...................................\\
           x_{n+m}=b_m-{a_{m1}x_1+a_{m2}x_2+...+a_{mn}x_n}\\
       \end{cases}
   \end{equation*}
    
&#8195;&#8195;то система ограничений преобразуется в систему уравнений:
    
   \begin{equation*}
       \begin{cases}
           {a_{11}x_1+a_{12}x_2+...+a_{1n}x_n}=b_1\\
           {a_{21}x_1+a_{22}x_2+...+a_{2n}x_n}=b_2\\
            ................................\\
           {a_{m1}x_1+a_{m2}x_2+...+a_{mn}x_n}=b_m
       \end{cases} 
   \end{equation*}
   
&#8195;&#8195; имеющую специальный вид. В этой системе каждая из переменных $x_{n+1}$, $x_{n+2}$, ..., $x_{n+m}$ исключена из всех уравнений, за исключением одного уравнения, в котором коэффициент при ней равен 1.

### Понятие опорного плана

- Базисным решением ОЗЛП называется такое решение системы уравнений, в котором все свободные переменные равны 0.
- Базисное решение ОЗЛП называется опорным решением (опорным планом), если в нем все базисные переменные неотрицательны.
- В теории симплекс-метода доказывается, что, если максимум целевой функции при данной в ОЗЛП системе ограничений существует, то он достигается на опорном решении.
- Опорное решение, на котором целевая функция достигает максимума, является оптимальным планом.

&#8195;&#8195; Замечание. Не следует думать, что ОЗЛП всегда имеет решение. Система ограничений, например, может быть противоречивой и задавать пустое множество решений. Система ограничений может также задавать неограниченное множество, на котором функция прибыли является неограниченной.

### Расчетный алгоритм симплекс-метода

&#8195;&#8195; Алгоритм симплекс-метода состоит из нескольких этапов: сначала происходит построение одного из опорных решений, а затем - «улучшение» этого решения, т.е. переход к другим опорным решениям, на которых значениецелевой функции не уменьшается. Для проведения расчетов используются симплекс-таблицы, составленные по коэффициентам системы, а процесс поиска и улучшения опорного решения заключается в пересчете элементов этих таблиц методом полных жордановых исключений.

### Разберем практическое решение

#### Шаг №1 Составление симплекс-таблицы

&#8195;&#8195; Составим первую симплекс-таблицу по системе и целевой функции.

&#8195;&#8195; Для каждого уравнения системы в первом столбце таблицы указывается соответствующая базисная переменная. Строка z называется целевой строкой.

Возможны два случая:

- Все числа $b_1$, $b_2$, ..., $b_m$ неотрицательны, т.е. решение является опорным. В этом случае строку $\varphi$ в таблице не заводим и сразу переходим ко 2-му этапу.

- Среди чисел $b_1$, $b_2$, ..., $b_m$ существуют отрицательные, т.е. решение не является опорным. Тогда для построения опорного решения запишем в строку $\varphi$ сумму тех строк таблицы, где стоят отрицательные значения $b_i$, а затем перейдем к анализу и улучшению решения по фиктивной целевой строке $\varphi$.

&#8195;&#8195; На основе введенных данных строится матрица, которая в последствии будет проверена на опорное решение.

&#8195;&#8195; В нашем случае эта матрица выглядит следующим образом:

In [3]:
from fractions import Fraction as Frc

l = []
B = []

for i in A:
    for j in i:
        l.append(Frc(j)) 
    B.append(l)
    l=[]
    
S = Simplex_table(B)
S.set_indexes(Col,Lin)
S.view()

*********************************************
     S_i0     x1       x3       x5       
-------------------------------------

x2     5        1        0        1      
-------------------------------------

x4     1        3        1        -1     
-------------------------------------

x6     5/2      -1/2     1        -1/2   
-------------------------------------

F      25       4        -5       5      
-------------------------------------

*********************************************


#### Шаг №2 Проверка на базисное решение.


In [4]:
S.find_base_solution()

----Поиск базисного решения----
*********************************************
     S_i0     x1       x3       x5       
-------------------------------------

x2     5        1        0        1      
-------------------------------------

x4     1        3        1        -1     
-------------------------------------

x6     5/2      -1/2     1        -1/2   
-------------------------------------

F      25       4        -5       5      
-------------------------------------

*********************************************
---Базисное решение:---
*********************************************
     S_i0     x1       x3       x5       
-------------------------------------

x2     5        1        0        1      
-------------------------------------

x4     1        3        1        -1     
-------------------------------------

x6     5/2      -1/2     1        -1/2   
-------------------------------------

F      25       4        -5       5      
-----------------------------------

#### Шаг №3 Поиск оптимального решения

&#8195;&#8195; Рассмотрим элементы целевой (фиктивной целевой) строки, стоящие в столбцах $x_1$, ..., $x_{n+m}$. Если среди них имеется хотя бы один отрицательный элемент, то решение надо улучшать по правилам этапа 3.
    
&#8195;&#8195; Если же все указанные элементы неотрицательны, то в случае, когда анализируется опорное решение по целевой строке z, это означает, что на этом решении функция z достигает максимума и нужно переходить к выписыванию соответствующего оптимального плана (этап 4).
    
&#8195;&#8195; В случае поиска опорного решения (анализ по фиктивной целевой строке $\varphi$) вся строка $\varphi$ должна состоять из нулей, иначе система ограничений противоречива, и ОЗЛП не имеет решения. Получение же нулевой строки $\varphi$ свидетельствует о том, что опорное решение построено. В этом случае фиктивная целевая строка удаляется из таблицы, а решение анализируется по целевой строке z.

&#8195;&#8195; Затем нам необходимо улучшать решение по целевой строке симплекс таблицы.

In [5]:
S.find_optimal_solution()

----Поиск оптимального решения----
*********************************************
     S_i0     x1       x3       x5       
-------------------------------------

x2     5        1        0        1      
-------------------------------------

x4     1        3        1        -1     
-------------------------------------

x6     5/2      -1/2     1        -1/2   
-------------------------------------

F      25       4        -5       5      
-------------------------------------

*********************************************
Целевой столбец: x1
Целевая строка: x4
*********************************************
     S_i0     x4       x3       x5       
-------------------------------------

x2     14/3     -1/3     -1/3     4/3    
-------------------------------------

x1     1/3      1/3      1/3      -1/3   
-------------------------------------

x6     8/3      1/6      7/6      -2/3   
-------------------------------------

F      71/3     -4/3     -19/3    19/3   
-----------------

#### Шаг №4 Запись оптимального плана

В плане указываем значения только исходных переменных $x_1$, $x_2$, ..., $x_n$, причем значения тех из них, которые являются свободными, полагаем равными нулю, а значения остальных переменных берем из второго столбца таблицы (столбца свободных членов). Из этого же столбца выписываем максимальное значение.

In [6]:
print("Оптимальное решение:")
print(" ")
print('Целевой столбец:', S.index_line[S.service_column])
print('Целевая строка:', S.index_column[S.service_line])
print(" ")
S.view()

Оптимальное решение:
 
Целевой столбец: x1
Целевая строка: x4
 
*********************************************
     S_i0     x1       x3       x2       
-------------------------------------

x5     5        1        0        1      
-------------------------------------

x4     6        4        1        1      
-------------------------------------

x6     5        0        1        1/2    
-------------------------------------

F      0        -1       -5       -5     
-------------------------------------

*********************************************


In [7]:
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import HTML

%matplotlib notebook

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Нажмите на кнопку чтобы увидеть код"></form>''')